In [11]:
import pandas as pd
from sqlalchemy import create_engine
from string import digits
import numpy as np
import os
import nltk
import string
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI


In [12]:
csv1 = "./indeed_job_dataset_jds_v2.csv"
# csv2 = "./indeed_job_dataset_V5_CSV.csv"
df1 = pd.read_csv(csv1, encoding = 'unicode_escape')
# df2 = pd.read_csv(csv2)
df1.head()
# df2.head()

,Index_No,Job_Title,Link,Queried_Salary,Job_Type,Skill,duplicate,Description,Unnamed: 8,Unnamed: 9,...,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']","[POSITION SUMMARY, The Business Analyst role i...","[position summary, the business analyst role i...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']","[What do we need?, \r\r\nYou to have an amazin...","[what do we need?, \r\r\nyou to have an amazin...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...","[Validate, analyze, and conduct statistical an...","[validate, analyze, and conduct statistical an...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],"[Full time, Washington, DC metro area, Startin...","[full time, washington, dc metro area, startin...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",[Assist in consultations with business partner...,[assist in consultations with business partner...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [13]:
df = df1[['Job_Type', 'Description']]
df.head()

,Job_Type,Description
0,data_scientist,"[position summary, the business analyst role i..."
1,data_scientist,"[what do we need?, \r\r\nyou to have an amazin..."
2,data_scientist,"[validate, analyze, and conduct statistical an..."
3,data_scientist,"[full time, washington, dc metro area, startin..."
4,data_scientist,[assist in consultations with business partner...


In [14]:

df.dropna(inplace=True)

len(df['Job_Type'])

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


5413

In [15]:
df.head()

,Job_Type,Description
0,data_scientist,"[position summary, the business analyst role i..."
1,data_scientist,"[what do we need?, \r\r\nyou to have an amazin..."
2,data_scientist,"[validate, analyze, and conduct statistical an..."
3,data_scientist,"[full time, washington, dc metro area, startin..."
4,data_scientist,[assist in consultations with business partner...


In [16]:
# df.drop_duplicates(inplace = True)

In [17]:
df.shape


(5413, 2)

In [18]:
#download stop words 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williammdavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [20]:
#show tokenization 
df['Description'].head().apply(process_text)

0    [position, summary, business, analyst, role, p...
1    [need, amazing, personality, communication, st...
2    [validate, analyze, conduct, statistical, anal...
3    [full, time, washington, dc, metro, area, star...
4    [assist, consultations, business, partners, in...
Name: Description, dtype: object

In [21]:
# #example 
# message4 = 'hello world hello hello world play'
# message5= 'test test test one hello'
# print(message4)
# print()

# #convert the text to a matrix of a token counts 
# from sklearn.feature_extraction.text import CountVectorizer
# bow4 =CountVectorizer(analyzer=process_text).fit_transform([[message4],[message5]])
# print(bow4)
# print(bow4.shape)

In [29]:
#convert collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer

message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['Description'])


In [31]:
# print(message_bow)

In [30]:
# word frequency-and word sgnificance 
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bow)
print(tfidf_transformer)

title_tfidf = tfidf_transformer.transform(message_bow)
print(title_tfidf)
print(title_tfidf.shape)


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
  (0, 33555)	0.06682020709161964
  (0, 33503)	0.107954082463933
  (0, 33493)	0.11310481652322983
  (0, 33174)	0.016510422907896255
  (0, 32899)	0.029509159440664342
  (0, 32694)	0.042913478039337934
  (0, 32490)	0.03275337876501664
  (0, 32461)	0.0380700962283523
  (0, 32174)	0.03657194591412919
  (0, 32120)	0.046789210045183216
  (0, 32066)	0.04409056332120154
  (0, 32014)	0.028164570879885897
  (0, 31978)	0.029553283393481344
  (0, 31806)	0.11310481652322983
  (0, 31804)	0.1664693082635492
  (0, 31468)	0.057842047209338526
  (0, 30975)	0.040997134753640654
  (0, 30808)	0.02874835941759064
  (0, 30759)	0.048204159438598196
  (0, 30473)	0.0832346541317746
  (0, 30356)	0.025918685310249726
  (0, 30239)	0.022911231384124504
  (0, 30167)	0.02452394268379129
  (0, 29900)	0.06474774276713924
  (0, 29819)	0.023829285448228932
  :	:
  (5412, 6001)	0.04132262028820754
  (5412, 5927)	0.07722274273021437
  (5412, 5875

In [32]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
model =  MultinomialNB().fit(title_tfidf, df['Job_Type'])

In [33]:
all_predictions = model.predict(title_tfidf)
print(all_predictions)

['data_scientist' 'data_scientist' 'data_scientist' ... 'data_engineer'
 'data_engineer' 'data_scientist']


In [44]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

confusion

Confusion Matrix: [[1010    9  655]
 [  44  620  664]
 [  17    2 2392]]


In [28]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df['Job_Type'],test_size=0.20, random_state=0)

In [29]:
message_bow.shape

(5413, 34069)

In [30]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier =  MultinomialNB().fit(X_train, y_train)

In [32]:
#print the prediction
print(classifier.predict(X_train))
#print values
print(y_train.values)

['data_analyst' 'data_engineer' 'data_scientist' ... 'data_scientist'
 'data_analyst' 'data_analyst']
['data_analyst' 'data_engineer' 'data_scientist' ... 'data_scientist'
 'data_analyst' 'data_analyst']


In [42]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_train)
print(classification_report(y_train, prediction))
print()

                precision    recall  f1-score   support

  data_analyst       0.89      0.88      0.88      1335
 data_engineer       0.93      0.90      0.92      1067
data_scientist       0.91      0.93      0.92      1928

      accuracy                           0.91      4330
     macro avg       0.91      0.91      0.91      4330
  weighted avg       0.91      0.91      0.91      4330




In [39]:
print('Confusion Matrix: \n', confusion_matrix(y_train,prediction))
print('Accuracy Matrix:',  accuracy_score(y_train,prediction))

Confusion Matrix: 
 [[1178   32  125]
 [  60  962   45]
 [  93   36 1799]]
Accuracy Matrix: 0.9096997690531178


In [41]:
#print the prediction
print(classifier.predict(X_test))
#print values
print(y_test.values)


['data_scientist' 'data_scientist' 'data_scientist' ... 'data_engineer'
 'data_engineer' 'data_scientist']
['data_scientist' 'data_scientist' 'data_scientist' ... 'data_engineer'
 'data_engineer' 'data_scientist']


In [43]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_test)
print(classification_report(y_test, prediction))
print()
print('Confusion Matrix: \n', confusion_matrix(y_test,prediction))
print()
print('Accuracy Matrix:',  accuracy_score(y_test,prediction))

                precision    recall  f1-score   support

  data_analyst       0.83      0.84      0.84       339
 data_engineer       0.89      0.85      0.87       261
data_scientist       0.88      0.88      0.88       483

      accuracy                           0.86      1083
     macro avg       0.86      0.86      0.86      1083
  weighted avg       0.86      0.86      0.86      1083


Confusion Matrix: 
 [[286  14  39]
 [ 16 223  22]
 [ 42  14 427]]

Accuracy Matrix: 0.8642659279778393
